# Split MISO into Sub-Areas
MISO: [Midcontinent Independent System Operator](https://www.misoenergy.org/)

We have contacted MISO people through their customer service to obtain geographical information on the *Local Resource Zone* (LRZ). Unfortunately, this information is confidential. We used Figure 1 from their public report available [here](https://www.purdue.edu/discoverypark/sufg/docs/publications/MISO%202016%20Independent%20Load%20Forecast%20Final.pdf) to perform a state to LRZ(s) mapping.

In [1]:
from functools import partial
import json
import os
import numpy as np
import pandas as pd
from powersimdata.network.usa_tamu.constants.zones import abv2state

Preliminary mapping of states to subareas.

In [2]:
state_to_miso_loadzones = {
    'North Dakota': 'LRZ1', 
    'Montana': 'LRZ1', 
    'South Dakota': 'LRZ1',
    'Minnesota': 'LRZ1',
    'Iowa': 'LRZ3_5', 
    'Missouri': 'LRZ3_5',
    'Wisconsin': 'LRZ2_7',
    'Michigan': 'LRZ2_7',
    'Illinois': 'LRZ4',
    'Indiana': 'LRZ6', 
    'Texas': 'LRZ8_9_10',
    'Louisiana': 'LRZ8_9_10',
    'Arkansas': 'LRZ8_9_10',
    'Alabama':'LRZ8_9_10',
    'Mississippi': 'LRZ8_9_10',
    'Kentucky':'LRZ6'}

Load the bus to BA mapping for the Eastern interconnect previously produced [here](../../eastern_demand_v5_demo/bus_ba_map.ipynb) and only consider entries for MISO.

In [3]:
bus_ba_map = pd.read_csv(os.path.join("..", "..", "eastern_demand_v5_demo/bus_ba_map.csv"), index_col=0)
bus_ba_map_miso = bus_ba_map[bus_ba_map['BA'] == 'MISO'].copy()

Create a new column that only contains the state abbrevation for the county.

In [4]:
bus_ba_map_miso['state'] = bus_ba_map_miso['County'].str.split('__').str[-1]
bus_ba_map_miso

,Pd,lat,lon,County,BA,zone_name,state
bus_id,,,,,,,
29396,15.37,30.445044,-88.409612,Jackson__MS,MISO,Alabama,MS
30015,30.56,31.540745,-91.381848,Adams__MS,MISO,Mississippi,MS
30017,30.56,31.519409,-91.305079,Adams__MS,MISO,Mississippi,MS
30018,30.56,31.523599,-91.358243,Adams__MS,MISO,Mississippi,MS
30019,30.56,31.442345,-91.296654,Adams__MS,MISO,Mississippi,MS
...,...,...,...,...,...,...,...
69946,1.70,48.307278,-106.272727,Valley__MT,MISO,Montana Eastern,MT
69948,0.85,48.222633,-106.316038,Valley__MT,MISO,Montana Eastern,MT
69949,0.66,48.932800,-106.642307,Valley__MT,MISO,Montana Eastern,MT


Map state column to MISO loadzones.

In [5]:
def get_miso_loadzone(abv2state, state_to_miso_loadzones, abv):
    mapped_zone = np.nan
    valid_abv = abv in abv2state.keys()
    if valid_abv:
        in_miso_mapping = abv2state[abv] in state_to_miso_loadzones
        if in_miso_mapping:
            mapped_zone =  state_to_miso_loadzones[abv2state[abv]]
    return mapped_zone

In [6]:
miso_loadzone = partial(get_miso_loadzone, abv2state, state_to_miso_loadzones)

Map state column to MISO loadzone

In [7]:
bus_ba_map_miso['miso_zone'] = bus_ba_map_miso.apply(lambda x: miso_loadzone(x['state']), axis=1)
bus_ba_map_miso

,Pd,lat,lon,County,BA,zone_name,state,miso_zone
bus_id,,,,,,,,
29396,15.37,30.445044,-88.409612,Jackson__MS,MISO,Alabama,MS,LRZ8_9_10
30015,30.56,31.540745,-91.381848,Adams__MS,MISO,Mississippi,MS,LRZ8_9_10
30017,30.56,31.519409,-91.305079,Adams__MS,MISO,Mississippi,MS,LRZ8_9_10
30018,30.56,31.523599,-91.358243,Adams__MS,MISO,Mississippi,MS,LRZ8_9_10
30019,30.56,31.442345,-91.296654,Adams__MS,MISO,Mississippi,MS,LRZ8_9_10
...,...,...,...,...,...,...,...,...
69946,1.70,48.307278,-106.272727,Valley__MT,MISO,Montana Eastern,MT,LRZ1
69948,0.85,48.222633,-106.316038,Valley__MT,MISO,Montana Eastern,MT,LRZ1
69949,0.66,48.932800,-106.642307,Valley__MT,MISO,Montana Eastern,MT,LRZ1


Check to make sure all states are mapped

In [8]:
bus_ba_map_miso[bus_ba_map_miso['miso_zone'].isnull()].state.unique()

array([], dtype=object)

Load in correction to state mapping using county-based map website

In [9]:
data = json.load(open('miso_counties.txt'))

Turn JSON structure into dictionary mapping

In [10]:
miso_minority_report = {}
for val in data['groups'].values():
    miso_minority_report[val['label']] = set(val['paths'])

Further correction to state mapping using county-based map data

In [11]:
miso_copy = bus_ba_map_miso.copy()

for zone, counties in miso_minority_report.items():
    for county in counties:
        miso_copy.loc[bus_ba_map_miso['County'] == county, 'miso_zone'] = zone
        
miso_copy

,Pd,lat,lon,County,BA,zone_name,state,miso_zone
bus_id,,,,,,,,
29396,15.37,30.445044,-88.409612,Jackson__MS,MISO,Alabama,MS,LRZ8_9_10
30015,30.56,31.540745,-91.381848,Adams__MS,MISO,Mississippi,MS,LRZ8_9_10
30017,30.56,31.519409,-91.305079,Adams__MS,MISO,Mississippi,MS,LRZ8_9_10
30018,30.56,31.523599,-91.358243,Adams__MS,MISO,Mississippi,MS,LRZ8_9_10
30019,30.56,31.442345,-91.296654,Adams__MS,MISO,Mississippi,MS,LRZ8_9_10
...,...,...,...,...,...,...,...,...
69946,1.70,48.307278,-106.272727,Valley__MT,MISO,Montana Eastern,MT,LRZ1
69948,0.85,48.222633,-106.316038,Valley__MT,MISO,Montana Eastern,MT,LRZ1
69949,0.66,48.932800,-106.642307,Valley__MT,MISO,Montana Eastern,MT,LRZ1


Output to disk

In [12]:
miso_copy.to_csv("bus_ba_map_miso.csv")